Prepared By [Izam Mohammed](https://github.com/izam-mohammed) 😊. Follow for more ❤️.

### GAN

In [17]:
!pip install -q nemo_toolkit['all'] huggingface

In [2]:
# Load FastPitch

from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.from_pretrained("nvidia/tts_en_fastpitch")

tts_en_fastpitch.nemo:   0%|          | 0.00/187M [00:00<?, ?B/s]

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
INFO:NeMo-text-processing:Creating ClassifyFst grammars.
[NeMo W 2024-02-21 19:17:06 en_us_arpabet:66] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2024-02-21 19:17:06 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
  

[NeMo I 2024-02-21 19:17:06 features:289] PADDING: 1
[NeMo I 2024-02-21 19:17:07 save_restore_connector:249] Model FastPitchModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--tts_en_fastpitch/snapshots/2c8305b7b41b33fd6367f0635796dc3a7a33cbf9/tts_en_fastpitch.nemo.


In [3]:
# Load vocoder

from nemo.collections.tts.models import HifiGanModel
model = HifiGanModel.from_pretrained(model_name="nvidia/tts_hifigan")

tts_hifigan.nemo:   0%|          | 0.00/315M [00:00<?, ?B/s]

[NeMo W 2024-02-21 19:17:11 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2024-02-21 19:17:11 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2024-02-21 19:17:11 features:289] PADDING: 0


[NeMo W 2024-02-21 19:17:11 features:266] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2024-02-21 19:17:11 features:289] PADDING: 0


[NeMo W 2024-02-21 19:17:11 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
      warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
    


[NeMo I 2024-02-21 19:17:12 save_restore_connector:249] Model HifiGanModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--tts_hifigan/snapshots/3ba1fed954276287015654bf4c78060ffc9a4772/tts_hifigan.nemo.


In [11]:
# generate audio

import soundfile as sf
parsed = spec_generator.parse("You can type your sentence here to get nemo to produce speech.")
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
audio = model.convert_spectrogram_to_audio(spec=spectrogram)

[NeMo W 2024-02-21 19:20:36 fastpitch:291] parse() is meant to be called in eval mode.
[NeMo W 2024-02-21 19:20:36 fastpitch:368] generate_spectrogram() is meant to be called in eval mode.


tensor([[ 0.0001, -0.0002, -0.0003,  ...,  0.0013,  0.0015,  0.0009]],
       grad_fn=<SqueezeBackward1>)

In [14]:
# Save the audio to disk in a file called speech.wav

import numpy as np
audio = audio.to('cpu').detach().numpy()

sf.write("speech.wav", np.ravel(audio), 22050)

In [15]:
from IPython.display import Audio, display

display(Audio("speech.wav", autoplay=True))

### Stable Diffusion

In [ ]:
!pip install diffusers["torch"] transformers

In [3]:
import torch
from diffusers import StableDiffusionPipeline

In [7]:
# Load the Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
prompt = "nice cars on speed"

In [9]:
# Number of images to generate (you can change this)
num_images = 1

# Generate images based on the prompt
images = pipe(prompt, num_images=num_images)

# Access the generated image
generated_image = images.images[0]

RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'

In [ ]:
# prompts

prompts = ["Alice in Wonderland, Ultra HD, realistic, futuristic, detailed, octane render, photoshopped, photorealistic, soft, pastel, Aesthetic, Magical background",
          "Anime style Alice in Wonderland, 90's vintage style, digital art, ultra HD, 8k, photoshopped, sharp focus, surrealism, akira style, detailed line art",
          "Beautiful, abstract art of Chesire cat of Alice in wonderland, 3D, highly detailed, 8K, aesthetic"]


images = []

In [ ]:
# running

for i, prompt in enumerate(prompts):
    image = pipe(prompt).images[0]
    image.save(f'picture_{i}.jpg')
    images.append(image)

In [ ]:
# showing
from IPython.display import Image

for j in range(i):
  display(Image(f'picture_{j}.png'))